In [ ]:
!pip install optuna

In [ ]:
!pip install neptune

In [ ]:
!pip install neptune_tensorflow_keras

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import drive
from keras.utils import to_categorical
from sklearn.metrics import f1_score, mean_squared_error
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from collections import Counter
import neptune
import tensorflow as tf
import optuna
from neptune.integrations.tensorflow_keras import NeptuneCallback
import itertools
from random import choice, randint

ModuleNotFoundError: No module named 'neptune'

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/loan_data.csv')

In [ ]:
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      45000 non-null  float64
 1   person_gender                   45000 non-null  object 
 2   person_education                45000 non-null  object 
 3   person_income                   45000 non-null  float64
 4   person_emp_exp                  45000 non-null  int64  
 5   person_home_ownership           45000 non-null  object 
 6   loan_amnt                       45000 non-null  float64
 7   loan_intent                     45000 non-null  object 
 8   loan_int_rate                   45000 non-null  float64
 9   loan_percent_income             45000 non-null  float64
 10  cb_person_cred_hist_length      45000 non-null  float64
 11  credit_score                    45000 non-null  int64  
 12  previous_loan_defaults_on_file  

In [ ]:
df_factorized = pd.DataFrame(df)
for column in df_factorized.select_dtypes(include=['object']).columns:
    df_factorized[column], unique = pd.factorize(df_factorized[column])

In [ ]:
X = df_factorized.drop('loan_status', axis=1).values
y = df_factorized['loan_status'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state=3, stratify=y)
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.fit_transform(X_test)

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [ ]:
run = neptune.init_run(
    project="YauheniMa/Parametrs-for-studing",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiOTAwOWNlMC0wNGM5LTQyZjEtOTMzMC04MmM3NTE4YjI4OWYifQ==",
)  # your credentials

def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
    optimizer = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])

    units_layer1 = trial.suggest_int('units_layer1', 32, 128)
    units_layer2 = trial.suggest_int('units_layer2', 32, 128)

    epochs = trial.suggest_int('epochs', 10, 20)

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units_layer1, activation='sigmoid', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(units_layer2, activation='sigmoid'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(2, activation='sigmoid')
    ])

    if optimizer == 'Adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['f1_score'])
    model_history = model.fit(X_train_norm, y_train_cat, epochs=epochs, batch_size=128, validation_data=(X_test_norm, y_test_cat))

    for epoch in range(epochs):
        run["train/loss"].log(model_history.history['loss'][epoch])
        m = model_history.history['f1_score'][epoch].numpy().tolist()              # Считаем f1_score вручную, т.к. history отдаёт голые значения
        mm = 2 * ((m[0] * m[1]) / (m[0] + m[1]))
        run["train/f1_score"].log(mm)
        run["validation/loss"].log(model_history.history['val_loss'][epoch])
        m = model_history.history['val_f1_score'][epoch].numpy().tolist()
        mm = 2 * ((m[0] * m[1]) / (m[0] + m[1]))
        run["validation/f1_score"].log(mm)

    return model_history.history['val_loss'][-1]


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

run.stop()

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/YauheniMa/Parametrs-for-studing/e/PAR-11


[I 2025-03-20 19:44:35,108] A new study created in memory with name: no-name-e9f2a66d-9cd4-4653-9d0e-fe0a6fb0d0e2


Epoch 1/15


<ipython-input-16-50ff1d0c005c>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)


282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - f1_score: 0.4381 - loss: 0.5463 - val_f1_score: 0.4375 - val_loss: 0.5139
Epoch 2/15
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - f1_score: 0.4386 - loss: 0.5135 - val_f1_score: 0.4375 - val_loss: 0.4868
Epoch 3/15
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - f1_score: 0.4391 - loss: 0.4852 - val_f1_score: 0.4375 - val_loss: 0.4322
Epoch 4/15
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - f1_score: 0.4995 - loss: 0.4284 - val_f1_score: 0.6342 - val_loss: 0.3548
Epoch 5/15
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - f1_score: 0.6989 - loss: 0.3553 - val_f1_score: 0.7772 - val_loss: 0.2970
Epoch 6/15
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - f1_score: 0.7771 - loss: 0.3038 - val_f1_score: 0.8217 - val_loss: 0.2684
Epoch 7/15
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - f1_score: 0.8122 - loss: 0.2784 - val_f1_score: 0.8342 - val_loss: 0.2540
Epoch 8/15
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - f1_score: 0.8153 - loss: 0.2711 - val_f1_score: 0.8393 - val_

...

Epoch 1/14
282/282 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - f1_score: 0.7449 - loss: 0.3609 - val_f1_score: 0.8238 - val_loss: 0.2298
Epoch 2/14
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - f1_score: 0.8440 - loss: 0.2266 - val_f1_score: 0.8453 - val_loss: 0.2078
Epoch 3/14
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - f1_score: 0.8481 - loss: 0.2197 - val_f1_score: 0.8595 - val_loss: 0.2005
Epoch 4/14
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - f1_score: 0.8539 - loss: 0.2097 - val_f1_score: 0.8633 - val_loss: 0.2019
Epoch 5/14
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - f1_score: 0.8598 - loss: 0.2073 - val_f1_score: 0.8544 - val_loss: 0.2089
Epoch 6/14
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.8582 - loss: 0.2113 - val_f1_score: 0.8700 - val_loss: 0.1992
Epoch 7/14
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - f1_score: 0.8565 - loss: 0.2100 - val_f1_score: 0.8691 - val_loss: 0.2042
Epoch 8/14
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - f1_score: 0.8611 - loss: 0.2051 - val_f1_score: 0.

[I 2025-03-20 20:08:11,492] Trial 48 finished with value: 0.19643697142601013 and parameters: {'learning_rate': 0.09233764007495027, 'optimizer': 'Adam', 'units_layer1': 63, 'units_layer2': 56, 'epochs': 14}. Best is trial 47 with value: 0.18236812949180603.


Epoch 1/13
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - f1_score: 0.7517 - loss: 0.3414 - val_f1_score: 0.8385 - val_loss: 0.2249
Epoch 2/13
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.8385 - loss: 0.2264 - val_f1_score: 0.8482 - val_loss: 0.2136
Epoch 3/13
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - f1_score: 0.8478 - loss: 0.2125 - val_f1_score: 0.8578 - val_loss: 0.2070
Epoch 4/13
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.8604 - loss: 0.2031 - val_f1_score: 0.8599 - val_loss: 0.2027
Epoch 5/13
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.8555 - loss: 0.2064 - val_f1_score: 0.8614 - val_loss: 0.1977
Epoch 6/13
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.8589 - loss: 0.1992 - val_f1_score: 0.8691 - val_loss: 0.1972
Epoch 7/13
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - f1_score: 0.8617 - loss: 0.1980 - val_f1_score: 0.8540 - val_loss: 0.1980
Epoch 8/13
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - f1_score: 0.8631 - loss: 0.1914 - val_f1_score: 0.

[I 2025-03-20 20:08:37,550] Trial 49 finished with value: 0.18615564703941345 and parameters: {'learning_rate': 0.027803307743689287, 'optimizer': 'Adam', 'units_layer1': 67, 'units_layer2': 61, 'epochs': 13}. Best is trial 47 with value: 0.18236812949180603.


[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 52 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 52 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/YauheniMa/Parametrs-for-studing/e/PAR-11/metadata


# Реализация класса дерева решений и рандомного леса

In [ ]:
class MyDecisionTree:


    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.best_split_value = None


    def entropy(self, y):
        class_counts = np.bincount(y)
        probabilities = class_counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 0.000000001))


    def information_gain(self, left_y, right_y, current_entropy):
        left_size = len(left_y)
        right_size = len(right_y)
        total_size = left_size + right_size

        left_entropy = self.entropy(left_y)
        right_entropy = self.entropy(right_y)

        gain = current_entropy - (left_size / total_size) * left_entropy - (right_size / total_size) * right_entropy
        return gain


    def fit(self):
        data = list(zip(self.X, self.y))
        data.sort(key=lambda x: x[0])

        best_split_value = None
        best_gain = -np.inf

        current_entropy = self.entropy(self.y)

        for i in range(1, len(data)):
            left_y = [label for _, label in data[:i]]
            right_y = [label for _, label in data[i:]]

            gain = self.information_gain(left_y, right_y, current_entropy)

            if gain > best_gain:
                best_gain = gain
                best_split_value = data[i-1][0]

        self.best_split_value = best_split_value


    def predict(self, X):
        return [1 if x >= self.best_split_value else 0 for x in X]


class MyRandomForest:


    def __init__(self, X, y, n_estimators=10):
        self.X = X
        self.y = y
        self.n_estimators = n_estimators

        self.trees = []


    def bootstrap_sample(self):
        indices = np.random.choice(len(self.X), len(self.X), replace=True)
        X_sample = [self.X[i] for i in indices]
        y_sample = [self.y[i] for i in indices]
        return X_sample, y_sample


    def fit(self):
        for _ in range(self.n_estimators):
            X_sample, y_sample = self.bootstrap_sample()
            tree = MyDecisionTree(X_sample, y_sample)
            tree.fit()
            self.trees.append(tree)


    def predict(self, X):
        all_predictions = np.array([tree.predict(X) for tree in self.trees])

        final_predictions = []
        for i in range(len(X)):
            votes = all_predictions[:, i]
            final_predictions.append(int(np.bincount(votes).argmax()))

        return final_predictions

In [ ]:
X = [ [1], [2], [3], [4]]
y = [1, 1, 0, 1]

forest = MyRandomForest(X, y, n_estimators=5)
forest.fit()

# Прогнозируем
predictions = forest.predict(X)
print(f"Прогнозы: {predictions}")

Прогнозы: [0, 1, 1, 1]
